# Consultant of Network Attack Detection


deskripsi (ada di pdf).

Network Intrusion Detection  System (NIDS)

## Goal

Untuk menganalisis klasifikasi type of attack berdasarkan variabel kunci.

Untuk menemukan model klasifikasi terbaik (objectives) Network Intrusion Detection  System (NIDS).

## Source of the dataset

GDrive: [click here](https://drive.google.com/drive/folders/1lOz8fx9AI7lWSEN9myIn18RmMrPErq8D?usp=drive_link)

## Atribute Information

The inputs (Var X/independent var) are as follows

- duration
- protocol_type
- service
- flag
- src_bytes
- dst_bytes
- land
- wrong_fragment
- urgent
- hot
- num_failed_logins
- logged_in
- num_compromised
- root_shell
- su_attempted
- num_root
- num_file_creations
- num_shells
- num_access_files
- num_outbound_cmds
- is_host_login
- is_guest_login
- count
- srv_count
- serror_rate
- srv_serror_rate
- rerror_rate
- srv_rerror_rate
- same_srv_rate
- diff_srv_rate
- srv_diff_host_rate
- dst_host_count
- dst_host_srv_count
- dst_host_same_srv_rate
- dst_host_diff_srv_rate
- dst_host_same_src_port_rate
- dst_host_srv_diff_host_rate
- dst_host_serror_rate
- dst_host_srv_serror_rate
- dst_host_rerror_rate
- dst_host_srv_rerror_rate
- type_of_attack

The output (target/Y/dependent var) is as follow:

- deteck attack

# Import Library


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

# from ydata_profiling import ProfileReport

# 1. Import Dataset

In [ ]:
# Path to training and validation dataset
train_PATH = 'D:\DAC-2023\Dataset\DataTrain_Preliminary.csv'
pred_PATH = 'D:\DAC-2023\Dataset\Data_Prediction.csv'

As a network attack detection consultant, we have been provided with two datasets: one for training and one for prediction. With these datasets, we can already narrow down the scope of our analysis, which in this case is creating a predictive model. Predictive model will naturally involve machine learning techniques. However, the field of machine learning itself is quite broad.

Therefore, we will begin by closely examining the data to refine our analysis to choose the best algorithm to discover the predictive model for effective network attack detection. This process involves exploring the data, identifying relevant features, and iteratively refining our approach until we find the most suitable predictive model.

In [ ]:
df_t = pd.read_csv(train_PATH, delimiter=';')
df_p = pd.read_csv(pred_PATH, delimiter=';')
df_t.head()

In [ ]:
df_t.shape

In [ ]:
train_cols = set(df_t.columns)
pred_cols = set(df_p.columns)

print("Columns in df_train but not in df_pred:", train_cols - pred_cols)
print("Columns in df_pred but not in df_train:", pred_cols - train_cols)

## Feature Description

- duration : the length of time connection in seconds
- protocol_type : the protocol used fot the connection
- service : the name of the network service on the target host
- flag : the status or state of the connection
- src_bytes : the number of bytes transferred from the source to destination
- dst_bytes : the number of bytes transferred from the destination to source
- land : indicates whether the connection is from/to the same host/port (1 if yes, 0 if no)
- wrong_fragment : the number of "wrong" fragments in a connection
- urgent : the number of urgent packets in a connection
- hot : the number of "hot" indicators in a connection
- num_failed_logins : the number of failed login attempts
- logged_in : indicates whether a login was successfully performed (1 if yes, 0 if no)
- num_compromised : the number of hosts compromised through the connection
- root_shell : indicates whether a root shell was obtained (1 if yes, 0 if no)
- su_attempted : indicates whether a "su root" command was attempted (1 if yes, 0 if no)
- num_root : the number of root access obtained
- num_file_creations : the number of file creation operations
- num_shells : te number of shell prompts
- num_access_files : the number of operations on access control files
- num_outbound_cmds : the number of outbound commands in an FTP session
- is_host_login : indicates whether the login belongs to the "hot" list (1 if yes, 0 if no)
- is_guest_login : indicates whether the login is a guest login (1 if yes, 0 if no)
- count : the number of connections to the same host as the current connection in the past two seconds
- srv_count : the number of connections to the same service as the current connection in the past two seconds
- serror_rate : the percentage of connections that had "SYN" errors (SYN : SYNCHRONIZE)
- srv_serror_rate : the percentage of connections to the same service that had "SYN" errors
- rerror_rate : the percentage of connections to the same service that had "REJ" errors (REJ : REJECTION) 
- srv_rerror_rate : the percentage of connections to the same 
service that had "REJ" errors
- same_srv_rate : the percentage of connections to the same service
- diff_srv_rate : the percentage of connections to different services
- srv_diff_host_rate : the percentage of connections to different hosts
- dst_host_count : the number of connections to the same destination host in the past two seconds
- dst_host_srv_count : the number of connections to the same destination service in the past two seconds
- dst_host_same_srv_rate : the percentage of connections to the same destination service
- dst_host_diff_srv_rate : the percentage of connections to different destination services
- dst_host_same_src_port_rate : he percentage of connections to the same destination host that originate from the same source port
- dst_host_srv_diff_host_rate : the percentage of connections to the same destination service but originating from different host
- dst_host_serror_rate : the percentage of connections to a specific destination host that have a “SYN” error
- dst_host_srv_serror_rate : the percentage of connections to a specific destination service that have a “SYN” error
- dst_host_rerror_rate : the percentage of connections to a specific destination host that have a “REJ” error
- dst_host_srv_rerror_rate : the percentage of connections to a specific destination service that have a “REJ” error
- type_of_attack : type of attack

# 2. Exploratory Data Analysis (EDA)

## Statistika umum

In [ ]:
df_t.describe()

## duration, src, dst

In [227]:
def filtered_by_type_of_attack(df, values):
    filtered = df[df['type_of_attack'] == values]
    return filtered

In [228]:
dos = filtered_by_type_of_attack(df_t, 'Denial of Service Attack')

In [229]:
dsd = dos[['duration', 'src_bytes', 'dst_bytes', 'type_of_attack']]
dsd.head()

,duration,src_bytes,dst_bytes,type_of_attack
10,0,54540,8314,Denial of Service Attack
14,0,28,0,Denial of Service Attack
56,9,54540,8314,Denial of Service Attack
91,1,54540,8314,Denial of Service Attack
258,0,1480,0,Denial of Service Attack


In [ ]:
dsd.

In [223]:
dsd.src_bytes.unique()

array(['54540', '28', '1480', '41772', '15492', '53452', '0', '51992',
       '22224', '53168', '50004', '46152', '45260', '47612', '564',
       '54020', '54060', '43232', '46720', '40636', '38284', '20440',
       '36256', '21048', '53208', '50816', '36500', '*', '18980', '45908',
       '33580', '16952', '52560', '13140', '50248', '13748', '51180',
       '24252', '29200', '38568', '49640', '37960', '53776', '51100',
       '16384'], dtype=object)

In [224]:
dsd.dst_bytes.unique()

array(['8314', '0', '2920', '7300', '8315', '4380', '1460', '28', '5840'],
      dtype=object)

In [232]:
# Create a boolean mask to identify rows with NaN values
nan_mask = dsd.isna().any(axis=1)

# Filter the DataFrame to show only rows with NaN values
rows_with_nan = dsd[nan_mask]

# Print the rows with NaNs
rows_with_nan

,duration,src_bytes,dst_bytes,type_of_attack
52524,0,NaN,8314,Denial of Service Attack


In [237]:
# show index 52524
dsd.loc[52524]

duration                                 0
src_bytes                              NaN
dst_bytes                             8314
type_of_attack    Denial of Service Attack
Name: 52524, dtype: object

In [ ]:
dsd.info()

In [ ]:
dsd.duration.unique()

In [ ]:
dsd.info()

In [239]:
# Replace the '*'s with NaNs
dsd = dsd.replace('*', np.nan)

In [ ]:
# Mengganti nilai "*" dengan  pada kolom 'duration', 'src_bytes', dan 'dst_bytes'
dsd['duration'] = dsd['duration'].replace('*', np.nan)
dsd['src_bytes'] = dsd['src_bytes'].replace('*', np.nan)
dsd['dst_bytes'] = dsd['dst_bytes'].replace('*', np.nan)

In [ ]:
# Mengubah tipe data menjadi numerik
dsd['duration'] = pd.to_numeric(dsd['duration'])
dsd['src_bytes'] = pd.to_numeric(dsd['src_bytes'])
dsd['dst_bytes'] = pd.to_numeric(dsd['dst_bytes'])

In [240]:
# merubah tipe data menjadi integer
dsd['duration'] = dsd['duration'].astype(int)
dsd['src_bytes'] = dsd['src_bytes'].astype(int)
dsd['dst_bytes'] = dsd['dst_bytes'].astype(int)

ValueError: cannot convert float NaN to integer

In [ ]:
# Menampilkan informasi DataFrame
dsd.info()

In [ ]:
dsd.src_bytes.unique()

In [233]:
# spline interpolation
dsd['duration'].interpolate(method= 'linear')
dsd['src_bytes'].interpolate(method='linear')
dsd['dst_bytes'].interpolate(method='linear')

10        8314
14           0
56        8314
91        8314
258          0
          ... 
112276    8314
112284       0
112355       0
112403       0
112411       0
Name: dst_bytes, Length: 1861, dtype: object

In [235]:
# fill missing value with spline interpolation
dsd['duration'].interpolate(inplace=True)
dsd['src_bytes'].interpolate(inplace=True)
dsd['dst_bytes'].interpolate(inplace=True)

C:\Users\gusti\AppData\Local\Temp\ipykernel_17980\210939014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsd['duration'].interpolate(inplace=True)
C:\Users\gusti\AppData\Local\Temp\ipykernel_17980\210939014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsd['src_bytes'].interpolate(inplace=True)
C:\Users\gusti\AppData\Local\Temp\ipykernel_17980\210939014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsd['dst_bytes'].i

In [ ]:
# check missing value
dsd.isnull().sum()

In [ ]:
dsd.describe()

In [ ]:
# boxplot
sns.boxplot(data=dsd)

## Checking data type

In [ ]:
# is_host_login type and unique value
unique_values_host_login = df_t['is_host_login'].unique()
print(unique_values_host_login)

# change to data type bool

In [ ]:
# is_guest_login type and unique value
unique_values_guest_login = df_t['is_guest_login'].unique()
print(unique_values_guest_login)

# change to data type bool

In [ ]:
# count data type and unique values
unique_values_count = df_t['count'].unique()
unique_values_count

# change to data type to int

In [ ]:
# srv_count data type and unique values
unique_values_srv_count = df_t['srv_count'].unique()
unique_values_srv_count

# change to data type to int

In [ ]:
# serror_rate data type and unique values
unique_values_serror_rate = df_t['serror_rate'].unique()
unique_values_serror_rate

# change to data type to float

In [ ]:
# srv_serror_rate data type and unique values
unique_values_srv_serror_rate = df_t['srv_serror_rate'].unique()
unique_values_srv_serror_rate

# change to data type to float

In [ ]:
# rerror_rate data type and unique values
unique_values_rerror_rate = df_t['rerror_rate'].unique()
unique_values_rerror_rate

# change to data type to float

In [ ]:
# srv_rerror_rate data type and unique values
unique_values_srv_rerror_rate = df_t['srv_rerror_rate'].unique()
unique_values_srv_rerror_rate

# change to data type to float

In [ ]:
# same_srv_rate data type and unique values
unique_values_same_srv_rate = df_t['same_srv_rate'].unique()
unique_values_same_srv_rate

# change to data type to float

In [ ]:
# diff_srv_rate data type and unique values
unique_values_diff_srv_rate = df_t['diff_srv_rate'].unique()
unique_values_diff_srv_rate

# change to data type to float

In [ ]:
# srv_diff_host_rate data type and unique values
unique_values_srv_diff_host_rate = df_t['srv_diff_host_rate'].unique()
unique_values_srv_diff_host_rate

# change to data type to float

In [ ]:
# dst_host_count data type and unique values
unique_values_dst_host_count = df_t['dst_host_count'].unique()
unique_values_dst_host_count

# change to data type to int

In [ ]:
# dst_host_srv_count data type and unique values
unique_values_dst_host_srv_count = df_t['dst_host_srv_count'].unique()
unique_values_dst_host_srv_count

# change to data type to int

In [ ]:
# dst_host_same_srv_rate data type and unique values
unique_values_dst_host_same_srv_rate = df_t['dst_host_same_srv_rate'].unique()
unique_values_dst_host_same_srv_rate

# change to data type to float

In [ ]:
# dst_host_diff_srv_rate data type and unique values
unique_values_dst_host_diff_srv_rate = df_t['dst_host_diff_srv_rate'].unique()
unique_values_dst_host_diff_srv_rate

# change to data type to float

In [ ]:
# dst_host_same_src_port_rate data type and unique values
unique_values_dst_host_same_src_port_rate = df_t['dst_host_same_src_port_rate'].unique()
unique_values_dst_host_same_src_port_rate

# change to data type to float

In [ ]:
# dst_host_srv_diff_host_rate data type and unique values
unique_values_dst_host_srv_diff_host_rate = df_t['dst_host_srv_diff_host_rate'].unique()
unique_values_dst_host_srv_diff_host_rate

# change to data type to float

In [ ]:
# dst_host_serror_rate data type and unique values
unique_values_dst_host_serror_rate = df_t['dst_host_serror_rate'].unique()
unique_values_dst_host_serror_rate

# change to data type to float

In [ ]:
# dst_host_srv_serror_rate data type and unique values
unique_values_dst_host_srv_serror_rate = df_t['dst_host_srv_serror_rate'].unique()
unique_values_dst_host_srv_serror_rate

# change to data type to float

In [ ]:
# dst_host_rerror_rate data type and unique values
unique_values_dst_host_rerror_rate = df_t['dst_host_rerror_rate'].unique()
unique_values_dst_host_rerror_rate

# change to data type to float

In [ ]:
# dst_host_srv_rerror_rate data type and unique values
unique_values_dst_host_srv_rerror_rate = df_t['dst_host_srv_rerror_rate'].unique()
unique_values_dst_host_srv_rerror_rate

# change to data type to float

In [ ]:
# type_of_attack data type and unique values
unique_values_type_of_attack = df_t['type_of_attack'].unique()
unique_values_type_of_attack

# change to data type to category

## Nilai 999 dan 99999

In [ ]:
# Mencari kolom-kolom yang mengandung nilai 999 atau 99999
kolom_dengan_999 = df_t.columns[df_t.eq(999).any()]
kolom_dengan_99999 = df_t.columns[df_t.eq(99999).any()]

# Menampilkan hasil
print("Kolom dengan nilai 999:")
print(kolom_dengan_999)

print("\nKolom dengan nilai 99999:")
print(kolom_dengan_99999)


In [ ]:
# Mencari kolom-kolom yang mengandung nilai 999 atau 99999
kolom_dengan_999 = df_p.columns[df_p.eq(999).any()]


## Perbandingan Persentase type of attack

In [ ]:
persentase = df_t.type_of_attack.value_counts(normalize=True) * 100
persentase

In [ ]:
# Mengelompokkan hasil berdasarkan aturan yang diberikan
grouped_results = {
    "Normal": persentase["normal"],
    "DoS": persentase["Denial of Service Attack"] + persentase["neptune"] + persentase["smurf"],
    "Probe": persentase["nmap"] + persentase["portsweep"] + persentase["satan"] + persentase["ipsweep"]
}

# Menampilkan hasil
for group, percentage in grouped_results.items():
    print(f"{group}: {percentage:.2f}%")


In [ ]:
# Menghitung jumlah Normal, DoS, dan Probe
jumlah_normal = df_t[df_t['type_of_attack'] == 'normal'].shape[0]
jumlah_dos = df_t[df_t['type_of_attack'].isin(['Denial of Service Attack', 'neptune', 'smurf'])].shape[0]
jumlah_probe = df_t[df_t['type_of_attack'].isin(['nmap', 'portsweep', 'satan', 'ipsweep'])].shape[0]

# Menampilkan hasil
print(f"Jumlah Normal: {jumlah_normal}")
print(f"Jumlah DoS: {jumlah_dos}")
print(f"Jumlah Probe: {jumlah_probe}")


## NaN and * value

In [ ]:

# Menghitung jumlah nilai NaN dalam DataFrame
jumlah_nan = df_t.isna().sum().sum()

# Menghitung jumlah nilai "*" dalam DataFrame
jumlah_asterisk = (df_t == '*').sum().sum()

# Menghitung total elemen dalam DataFrame
total_elemen = df_t.size

# Menghitung persentase NaN dan "*" dalam DataFrame
persentase_nan = (jumlah_nan / total_elemen) * 100
persentase_asterisk = (jumlah_asterisk / total_elemen) * 100

# Menampilkan hasil
print(f"Jumlah NaN: {jumlah_nan}")
print(f"Jumlah '*': {jumlah_asterisk}")
print(f"Persentase NaN: {persentase_nan:.2f}%")
print(f"Persentase '*': {persentase_asterisk:.2f}%")

## Fitur terkait keamanan

In [ ]:
k_secure = df_t[['num_failed_logins', 'num_access_files', 
                 'num_compromised', 'root_shell', 
                 'su_attempted', 'type_of_attack']]
k_secure.head()

In [ ]:
k_secure.info()

In [ ]:
k_secure_sorted1 = k_secure.sort_values(by='num_failed_logins', ascending=False)
k_secure_sorted1.head()

In [ ]:
k_secure_sorted2 = k_secure.sort_values(by='num_access_files', ascending=False)
k_secure_sorted2.head()

In [ ]:
k_secure_sorted3 = k_secure.sort_values(by='num_compromised', ascending=False)
k_secure_sorted3.head()

In [ ]:
k_secure_sorted4 = k_secure.sort_values(by='root_shell', ascending=False)
k_secure_sorted4.head()

In [ ]:
k_secure_sorted5 = k_secure.sort_values(by='su_attempted', ascending=False)
k_secure_sorted5.head()

In [ ]:
k_secure.describe()

In [ ]:
k_secure['num_failed_logins'].value_counts().sort_index()

## FItur Terkait koneksi

In [ ]:
c_conection = df_t[['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'type_of_attack']]
c_conection.head()

In [ ]:
c_conection.describe()

In [ ]:
c_conection.info()

## Fitur Error Rate

In [ ]:
r_error = df_t[['serror_rate', 'rerror_rate', 
                'srv_serror_rate', 'srv_rerror_rate', 
                'dst_host_serror_rate', 'dst_host_rerror_rate', 'type_of_attack']]
r_error.head() 

In [ ]:
r_error.describe()

In [ ]:
r_error.info()

In [ ]:
r_error_g = r_error.groupby(by='type_of_attack')['serror_rate'].count().sort_values(ascending=False)
r_error_g

## Fitur Terkait Host (BERANTAKAN)

Clean wkwk hapus "*" value dan hapus "99999" value

In [ ]:
host = df_t[['dst_host_count', 'dst_host_srv_count', 
             'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'type_of_attack']]
host.head()

In [ ]:
# Membuat bubble plot hanya untuk data yang memiliki dst_host_count dan dst_host_srv_count == '99999'
filtered_host = host[(host['dst_host_srv_count'] == '99999')]

In [ ]:
filtered_host

In [ ]:
fgrup = filtered_host.groupby(by='type_of_attack')['dst_host_srv_count'].count().sort_values(ascending=False)
fgrup

In [ ]:
host.head().sort_values(by='dst_host_count', ascending=False)

In [ ]:
host.head().sort_values(by='dst_host_srv_count', ascending=False)  

In [ ]:
host.head().sort_values(by='dst_host_same_srv_rate', ascending=False)

In [ ]:
host.sort_values(by='type_of_attack', ascending=True)

In [ ]:
host.info()

In [ ]:
host['dst_host_count'] = pd.to_numeric(host['dst_host_count'])
host['dst_host_srv_count'] = pd.to_numeric(host['dst_host_srv_count'])

In [ ]:
host.describe()

In [ ]:
invalid_values = host[host['dst_host_count'] == '*'].count()
invalid_values

In [ ]:
invalid_values2 = host[host['dst_host_count'] == '99999'].count()
invalid_values2

In [ ]:
host = host[host['dst_host_count'] != '*']
host = host[host['dst_host_srv_count'] != '*']

In [ ]:
host = host[host['dst_host_count'] != '99999']
host = host[host['dst_host_srv_count'] != '99999']

In [ ]:
host = host[host['type_of_attack'] != 'Normal']

In [ ]:
login = df_t[['dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',]]
login.head()

In [ ]:
# Hitung Q1, Q3, dan IQR
Q1 = host['dst_host_count'].quantile(0.25)
Q3 = host['dst_host_count'].quantile(0.75)
IQR = Q3 - Q1

# Hitung Batas Atas dan Batas Bawah
Upper_Bound = Q3 + 1.5 * IQR
Lower_Bound = Q1 - 1.5 * IQR

# Mengecualikan data outlier
filtered_host = host[(host['dst_host_count'] >= Lower_Bound) & (host['dst_host_count'] <= Upper_Bound)]

In [ ]:
# Membuat sebuah figure dengan subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

# Membuat violin plot untuk kolom dst_host_count
sns.violinplot(x='dst_host_count', data=host, ax=axes[0, 0])
axes[0, 0].set_title('Violin Plot - dst_host_count')

# Membuat violin plot untuk kolom dst_host_srv_count
sns.violinplot(x='dst_host_srv_count', data=host, ax=axes[0, 1])
axes[0, 1].set_title('Violin Plot - dst_host_srv_count')

# Membuat violin plot untuk kolom dst_host_same_srv_rate
sns.violinplot(x='dst_host_same_srv_rate', data=host, ax=axes[1, 0])
axes[1, 0].set_title('Violin Plot - dst_host_same_srv_rate')

# Membuat violin plot untuk kolom dst_host_diff_srv_rate
sns.violinplot(x='dst_host_diff_srv_rate', data=host, ax=axes[1, 1])
axes[1, 1].set_title('Violin Plot - dst_host_diff_srv_rate')

# Mengatur tata letak subplot
plt.tight_layout()

# Menampilkan plot
plt.show()


In [ ]:
# Konversi kolom dst_host_count menjadi tipe data numerik
host['dst_host_count'] = pd.to_numeric(host['dst_host_count'], errors='coerce')

# Membuat sebuah figure
plt.figure(figsize=(8, 6))

# Membuat boxplot untuk kolom dst_host_count
sns.boxplot(x='dst_host_count', data=host)
plt.title('Box Plot - dst_host_count')

# Menampilkan plot
plt.show()

In [ ]:
# Membuat sebuah figure
plt.figure(figsize=(8, 6))

# Membuat scatter plot untuk kolom dst_host_count dan dst_host_srv_count
sns.scatterplot(x='dst_host_count', y='dst_host_srv_count', data=host)
plt.title('Scatter Plot - dst_host_count vs dst_host_srv_count')

# Menampilkan plot
plt.show()

In [ ]:
# Membuat bubble plot
plt.figure(figsize=(10, 8))
sns.scatterplot(x='dst_host_count', y='dst_host_srv_count', size='dst_host_same_srv_rate', data=host, sizes=(50, 500))
plt.title('Bubble Plot - dst_host_count vs dst_host_srv_count')
plt.xlabel('dst_host_count')
plt.ylabel('dst_host_srv_count')

# Menampilkan plot
plt.show()

## Fitur Terkait Distribusi Port

In [ ]:
port = df_t[['dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'service', 'type_of_attack']]

In [ ]:
port.head()

In [ ]:
port.info()

In [ ]:
# grupby dst_host_srv_diff_host_rate
p_g1 = port.groupby(by='dst_host_srv_diff_host_rate')['dst_host_same_src_port_rate'].count().sort_values(ascending=False)
p_g1

In [ ]:
# grupby dst_host_srv_diff_host_rate
p_g2 = port.groupby(by='dst_host_srv_diff_host_rate')['dst_host_srv_diff_host_rate'].count().sort_values(ascending=False)
p_g2

In [ ]:
# menghapus nilai * pada kolom dst_host_same_src_port_rate dan dst_host_srv_diff_host_rate
port = port[port['dst_host_same_src_port_rate'] != '*']
port = port[port['dst_host_srv_diff_host_rate'] != '*']

In [ ]:
# menghapus nilai 99999 pada kolom dst_host_same_src_port_rate dan dst_host_srv_diff_host_rate
port = port[port['dst_host_same_src_port_rate'] != '99999']
port = port[port['dst_host_srv_diff_host_rate'] != '99999']

In [ ]:
# merubah tipe data dst_host_same_src_port_rate dan dst_host_srv_diff_host_rate menjadi numerik
port['dst_host_same_src_port_rate'] = pd.to_numeric(port['dst_host_same_src_port_rate'])
port['dst_host_srv_diff_host_rate'] = pd.to_numeric(port['dst_host_srv_diff_host_rate'])

In [ ]:
port_x = port

In [ ]:
# Specify a list of markers for each unique type_of_attack
markers = ["o", "s", "D", "X", "P", "*", "^", "v"]

# Create a pair plot
sns.pairplot(port_x, hue='type_of_attack', diag_kind='kde', markers=markers, palette='Set1')
plt.suptitle('Pair Plot - Persebaran dan Korelasi Data', y=1.02)

# Show the plot
plt.show()

# X ***Kategori Variabel***

In [ ]:
k_vars = df_t[['protocol_type', 'service', 'flag']]
k_vars.head()

## **Looking at Type of Attack**

label column : type of attack

In [ ]:
attack_type = df_train['type_of_attack'].unique()

print("Nilai yang ada dalam kolom x3:")
print(attack_type)

TYPE OF ATTACK :
- nmap
- neptune
- normal
- Denial of Service Attack
- portsweep
- satan
- ipsweep
- smurf

Normal

DOS ATTACK: 
neptune -> working to SYN flood Denial of Service on one or more ports, Neptune attack make the memory resources busy by sending TCP packet requesting to initiate a TCP session,
DOS ->  makes the computer system to be hectic by sending many ping, 
Smurf -> Denial of Service ICMP echo reply flood. 

PROBE: nmap, portsweep, satan, ipsweep, 

This paper focused on Neptune and Smurf types of attack. Those attack characterized as DoS Attack. Dos Attack makes the computer system to be hectic by sending many ping. Neptune attack working to SYN flood Denial of Service on one or more ports while the function of Smurf attack is Denial of Service ICMP echo reply flood. Neptune attack make the memory resources busy by sending TCP packet requesting to initiate a TCP session.  After many TCP packet sent, the computer system finally runs out of memory resources. Smurf attack is the popular type of attack sending ICMP echo request packet to middle device. ICMP packets have source address name as victim’s IP address and middle device address as destination address. DoS attack reduces the performance of server by overflowing the ICMP traffic as discussed above in the Neptune and Smurf attack. So, detection of DoS attack is very significant to protect the system. 

KOLOM KUNCI:
- protocol type
- port
- 

In [ ]:
def filtered_by_type_of_attack(df, values):
    filtered = df[df['type_of_attack'] == values]
    return filtered

In [ ]:
filtered_by_type_of_attack(df_train, 'nmap')

In [ ]:
ProfileReport(filtered_by_type_of_attack(df_train, 'nmap'))

In [ ]:
filtered_by_type_of_attack(df_train, 'neptune')

In [ ]:
filtered_by_type_of_attack(df_train, 'normal')

In [ ]:
filtered_by_type_of_attack(df_train, 'Denial of Service Attack')

In [ ]:
filtered_by_type_of_attack(df_train, 'portsweep')

In [ ]:
filtered_by_type_of_attack(df_train, 'satan')

In [ ]:
filtered_by_type_of_attack(df_train, 'ipsweep')

In [ ]:
filtered_by_type_of_attack(df_train, 'smurf')

## **Duplicate Data**

In [ ]:
duplicates = df_train[df_train.duplicated()]

In [ ]:
duplicates

is it really duplicate? harus nemu variabel kunci yg menentukan itu row duplicate or no. atau jangan jangan nilai duplicate ini bisa jadi kunci network attack (karena patternnya sama)